### 2. Создание и обработка форм

#### 2.1 Создание простой формы с полями для ввода данных

Чтобы создать форму в Django, нужно создать новый файл внутри приложения или обновить существующий. Стандартно файл с формами называют **`forms.py`** и размещают его в корне вашего приложения (например, если ваше приложение называется `myapp`, файл будет в директории `myapp/forms.py`).

##### Пример создания формы:

1. **Создаём файл `forms.py`:**

    Если у вас его ещё нет, создайте файл в корне приложения:
    ```
    myapp/
    ├── forms.py
    ├── models.py
    ├── views.py
    └── ...
    ```

2. **Импортируем необходимые классы:**
   
    В `forms.py` нужно импортировать базовый класс формы `forms.Form` и соответствующие поля из модуля `django.forms`:
    ```python
    from django import forms
    ```

3. **Создаём класс формы:**

    В этом классе мы определим поля, которые пользователь будет заполнять. В качестве примера создадим форму для обратной связи, в которой пользователь вводит своё имя, email и сообщение.

    ```python
    class ContactForm(forms.Form):
        name = forms.CharField(label='Ваше имя', max_length=100)
        email = forms.EmailField(label='Ваш email')
        message = forms.CharField(widget=forms.Textarea, label='Сообщение')
    ```

    В этом примере:
    - Поле `name` — это текстовое поле с меткой "Ваше имя".
    - Поле `email` — специальное поле для ввода email-адреса, которое автоматически проверяет правильность формата.
    - Поле `message` — это поле с виджетом `Textarea`, чтобы пользователь мог ввести длинное сообщение.


#### 2.2 Валидация данных формы

После того как форма создается, её основная задача — проводить валидацию данных, которые пользователь вводит. Django уже включает встроенную валидацию для стандартных полей, но вы можете добавлять и собственную.

##### Встроенная валидация:

Django автоматически проверяет, например:
- что обязательные поля не остаются пустыми;
- что данные соответствуют типам (например, email имеет корректный формат).

Для того чтобы проверить, прошла ли форма валидацию, используется метод `is_valid()`, который возвращает `True`, если все поля заполнены корректно.

Пример использования валидации:
```python
form = ContactForm(request.POST)
if form.is_valid():
    # Данные корректны, можно их обрабатывать
    name = form.cleaned_data['name']
    email = form.cleaned_data['email']
    message = form.cleaned_data['message']
else:
    # Форма содержит ошибки, можно их вывести
    print(form.errors)
```

##### Кастомная валидация:

Если вам нужно провести дополнительную проверку данных, вы можете переопределить метод `clean()` или методы для отдельных полей, например `clean_email()`. 

Пример кастомной валидации для поля `email`:
```python
class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    message = forms.CharField(widget=forms.Textarea)

    def clean_email(self):
        email = self.cleaned_data.get('email')
        if not email.endswith('@example.com'):
            raise forms.ValidationError('Email должен быть с доменом @example.com')
        return email
```

Метод `clean_email()` проверяет, что введённый email заканчивается на `@example.com`, и если это не так, добавляет сообщение об ошибке.


#### 2.3 Обработка данных формы на стороне сервера

Когда данные формы валидированы, следующим шагом является их обработка. В Django обработка формы происходит в **views**, обычно в виде функции или класса.

##### Пример обработки формы в представлении:

1. Откройте или создайте файл **`views.py`** в вашем приложении.

2. Импортируйте вашу форму и другие необходимые компоненты:
    ```python
    from django.shortcuts import render
    from .forms import ContactForm
    ```

3. Создайте представление для обработки формы. Например, представление на основе функции:
    ```python
    def contact_view(request):
        if request.method == 'POST':
            form = ContactForm(request.POST)
            if form.is_valid():
                # Здесь можно обрабатывать данные формы
                name = form.cleaned_data['name']
                email = form.cleaned_data['email']
                message = form.cleaned_data['message']
                # Например, отправить email или сохранить данные в базу
                # После обработки можно перенаправить пользователя на другую страницу
                return render(request, 'myapp/success.html')
        else:
            form = ContactForm()

        return render(request, 'myapp/contact.html', {'form': form})
    ```

В этом примере:
- Если метод запроса POST, значит, форма отправлена, и мы проверяем её валидацию с помощью `is_valid()`.
- Если данные валидны, мы их извлекаем через `form.cleaned_data` и можем с ними работать.
- Если данные формы некорректны или страница загружена впервые (метод запроса GET), форма рендерится без данных.


#### 2.4 Вывод ошибок валидации

Когда форма содержит ошибки, они должны быть выведены пользователю для исправления. Django автоматически генерирует сообщения об ошибках и предоставляет их через форму.

##### Пример вывода ошибок в шаблоне:

1. Откройте или создайте файл шаблона для формы, например **`templates/myapp/contact.html`**.

2. В шаблоне используйте следующие теги для отображения формы и её ошибок:
    ```html
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        {% for field in form %}
            <div>
                <label>{{ field.label_tag }}</label>
                {{ field }}
                {% if field.errors %}
                    <ul class="errorlist">
                        {% for error in field.errors %}
                            <li>{{ error }}</li>
                        {% endfor %}
                    </ul>
                {% endif %}
            </div>
        {% endfor %}
        <button type="submit">Отправить</button>
    </form>
    ```

В этом шаблоне:
- Используется тег `{% csrf_token %}`, чтобы защитить форму от CSRF-атак.
- Поля формы выводятся с помощью цикла, и для каждого поля проверяются ошибки. Если ошибки есть, они отображаются в виде списка под соответствующим полем.



##### Стандартные ошибки:

Django автоматически выводит стандартные сообщения об ошибках для каждого поля, например:
- "Это поле обязательно" (если поле является обязательным).
- "Введите корректный email" (если email введён неверно).

##### Кастомные сообщения:

**Если вы добавляли собственные сообщения об ошибках, они также будут выведены рядом с полем, для которого они были определены.**